### CAR PRICE PREDICTION

### Problem Statement:
    
**With the covid 19 impact in the market, we have seen lot of changes in the car market. Now some
cars are in demand hence making them costly and some are not in demand hence cheaper. One
of our clients works with small traders, who sell used cars. With the change in market due to covid
19 impact, our client is facing problems with their previous car price valuation machine learning
models. So, they are looking for new machine learning models from new data. We have to make
car price valuation model.**

Aim : Scrape at least 5000 used car data from websites(Olx, Cardekho, Cars24 etc.) for different locations. 

Important data to be scraped are Brand, model, variant, manufacturing year, driven kilometers, fuel, number of owners, location and
at last target variable Price of the car. Try to include all types of cars in your data for example- SUV, Sedans, Coupe, minivan,
Hatchback.

In [178]:
#Importing the libraries

import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


import pandas as pd
import re
import time

import warnings
warnings.filterwarnings('ignore')

We are collecting the data from the below popular cities:
    
`New Delhi`
`Noida`
`Gurgaon`
`Mumbai`
`Pune`
`Bengaluru`
`Hyderabad`
`Chennai`
`Kerala`
`Ahmedabad`

Details to be scraped:

`Brand`
`Model`
`Manufacturing year`
`Driven km`
`Fuel type`
`Number of owners`
`Location`
`Car price`

We are going to collect details of 200 car details for each cities from three websites.

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
driver.maximize_window()
time.sleep(1)

#### Collecting Used Car Data From OLX.in

In [3]:
Locations = ['Delhi', 'Kerala', 'Gurgaon', 'Mumbai', 'Pune', 'Bengaluru', 'Hyderabad', 'Chennai', 'Kolkata', 'Ahmedabad']
url_olx = []

for i in Locations:
    
    driver.get("https://www.olx.in")
    time.sleep(2)
    
    #Launching the page "Cars"
    cars_page = driver.find_element_by_xpath("//div[@id = 'container']/div[1]/div/div/div[1]/div[2]/div[1]/a/span")
    cars_page.click()
    time.sleep(1)

    location_input = driver.find_element_by_xpath("//div[@id = 'container']/header/div/div/div[2]/div/div/div[1]/div/div[1]/input")
    location_input.click()

    location_input.send_keys(i)
    time.sleep(3)

    location_click = driver.find_element_by_xpath("//div[@id = 'container']/header/div/div/div[2]/div/div/div[1]/div/div[2]/div/div/div[1]/div/span")
    location_click.click()

    #Clicking the load more button to get required number of data from the websites
    url_loc = []
    while len(url_loc)<200:
        url_loc = []
        
        try:
            time.sleep(3)
            load_more = driver.find_element_by_xpath("//button[@data-aut-id = 'btnLoadMore']")
            load_more.click()
        except NoSuchElementException as e:
            height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, {})".format(height-2000))
            load_more = driver.find_element_by_xpath("//button[@data-aut-id = 'btnLoadMore']")
            load_more.click()
        
        time.sleep(2)
        url_tags = driver.find_elements_by_xpath("//ul[@class = 'c1gK9 _1Xvo7']/li/a")
        for ur in url_tags:
            url_loc.append(ur.get_attribute('href'))
    url_loc = url_loc[:200]    
    url_olx.extend(url_loc)

    
print(len(url_olx))

2000


We have scraped the URLs for 2000 cars for different locations from Olx.in. Now we can scrape the required information for each cars from the URLs.

In [4]:
driver.close()

In [5]:
brand_olx = []
model_olx = []
man_year_olx = []
driven_km_olx = []
fuel_type_olx = []
owners_olx = []
car_loc_olx = []
car_price_olx = []

brand_olx_temp = []
model_olx_temp = []
man_year_olx_temp = []
driven_km_olx_temp = []
fuel_type_olx_temp = []
owners_olx_temp = []
car_loc_olx_temp = []
car_price_olx_temp = []

In [13]:
delay = 10  #seconds



def olx_temp(a,b):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")

    driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
    driver.maximize_window()
    time.sleep(1)
    global brand_olx_temp, model_olx_temp, man_year_olx_temp, driven_km_olx_temp, fuel_type_olx_temp, owners_olx_temp, car_loc_olx_temp, car_price_olx_temp
    brand_olx_temp = []
    model_olx_temp = []
    man_year_olx_temp = []
    driven_km_olx_temp = []
    fuel_type_olx_temp = []
    owners_olx_temp = []
    car_loc_olx_temp = []
    car_price_olx_temp = []
    
    for i in url_olx[a:b]:
        if len(car_price_olx_temp)%50 ==0:
            driver.close()

            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument("--incognito")

            driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
            driver.maximize_window()
            time.sleep(2)
            

        try:
            try:
                time.sleep(3)
                driver.get(i)
            except (WebDriverException, TimeoutException) as e:
                WebDriverWait(driver,2)
                driver.get(i)
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
                driver.execute_script("window.scrollTo(0, 500)")
                WebDriverWait(driver,1)
        except (WebDriverException) as e:
            WebDriverWait(driver,2)
            driver.get(i)
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
            driver.execute_script("window.scrollTo(0, 500)")
            WebDriverWait(driver,1)

        try:
            try:
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
                brand_tag = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div > div > div._35xN1").text
                keyword = re.compile(r"([A-Za-z]+) ")
                brand_name = []
                brand_name = keyword.findall(str(brand_tag))
                br = []
                for i in brand_name:
                    br.extend(i.replace(" ","").split(","))
                brand_olx_temp.append(br[0])
            except (NoSuchElementException,TimeoutException) as e:
                brand_olx_temp.append("--")
        except WebDriverException as e:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
            brand_tag = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div > div > div._35xN1").text
            keyword = re.compile(r"([A-Za-z]+) ")
            brand_name = []
            brand_name = keyword.findall(str(brand_tag))
            br = []
            for i in brand_name:
                br.extend(i.replace(" ","").split(","))
            brand_olx_temp.append(br[0])


        try:
            try:
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
                model_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div > div > div._35xN1").text
                keyword = re.compile(r"\s+[\(0-9]+\)")
                model_name = keyword.sub("",str(model_tags))
                model_olx_temp.append(model_name.replace(br[0],"")[1:])
            except (NoSuchElementException,TimeoutException) as e:
                model_olx_temp.append("--")
        except WebDriverException as e:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
            model_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div > div > div._35xN1").text
            keyword = re.compile(r"\s+[\(0-9]+\)")
            model_name = keyword.sub("",str(model_tags))
            model_olx_temp.append(model_name.replace(br[0],"")[1:])


        try:
            try:
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
                man_year_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div > div > div._35xN1").text
                keyword = re.compile(r"([\(]+[0-9]+)")
                man_year_name = keyword.finditer(str(man_year_tags))
                for i in man_year_name:
                    man_year_olx_temp.append(i.group(0).replace("(",""))
            except (NoSuchElementException,TimeoutException) as e:
                man_year_olx_temp.append("--")
        except WebDriverException as e:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
            man_year_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div > div > div._35xN1").text
            keyword = re.compile(r"([\(]+[0-9]+)")
            man_year_name = keyword.finditer(str(man_year_tags))
            for i in man_year_name:
                man_year_olx_temp.append(i.group(0).replace("(",""))
  
        try:
            try:
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
                driven_km_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div._3B8tc.i91uu > div > div.aOxkz > div:nth-child(2) > div").text
                driven_km_olx_temp.append(driven_km_tags.replace(" KM",""))
            except (NoSuchElementException,TimeoutException) as e:
                driven_km_olx_temp.append("--")
        except WebDriverException as e:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
            driven_km_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div._3B8tc.i91uu > div > div.aOxkz > div:nth-child(2) > div").text
            driven_km_olx_temp.append(driven_km_tags.replace(" KM",""))

        
        try:
            try:
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
                fuel_type_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div > div > div.aOxkz > div:nth-child(1) > div").text
                fuel_type_olx_temp.append(fuel_type_tags)
            except (NoSuchElementException,TimeoutException) as e:
                fuel_type_olx_temp.append("--")
        except WebDriverException as e:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
            fuel_type_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(1) > div > div > div.aOxkz > div:nth-child(1) > div").text
            fuel_type_olx_temp.append(fuel_type_tags)


        try:
            try:
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
                owner_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(2) > div > div:nth-child(3) > div > div:nth-child(1) > div._1idEV > div._1gasz").text
                owners_olx_temp.append(owner_tags)
            except (NoSuchElementException,TimeoutException) as e:
                owners_olx_temp.append("--")
        except WebDriverException as e:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
            owner_tags = driver.find_element_by_css_selector("div.sE139 > div:nth-child(2) > div > div:nth-child(3) > div > div:nth-child(1) > div._1idEV > div._1gasz").text
            owners_olx_temp.append(owner_tags)


        try:
            try:
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
                car_loc_tag = driver.find_element_by_css_selector("div._3S9Vi._1tOAI > div._1cMxH > div.BfeRg > div._5iGXa.mwOt2 > div._2flnz > div.ZGU9S > div").text
                car_loc_olx_temp.append(car_loc_tag)
            except (NoSuchElementException,TimeoutException) as e:
                car_loc_olx_temp.append("--")
        except WebDriverException as e:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
            car_loc_tag = driver.find_element_by_css_selector("div._3S9Vi._1tOAI > div._1cMxH > div.BfeRg > div._5iGXa.mwOt2 > div._2flnz > div.ZGU9S > div").text
            car_loc_olx_temp.append(car_loc_tag)

        try:
            try:
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
                car_price_tag = driver.find_element_by_css_selector("div._1cMxH > div._1Jl6c > div > div._3FkyT").text
                car_price_olx_temp.append(car_price_tag.replace("₹ ","").replace(",",""))
            except (NoSuchElementException,TimeoutException) as e:
                car_price_olx_temp.append("--")
        except WebDriverException as e:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'section:nth-child(2)')))
            car_price_tag = driver.find_element_by_css_selector("div._1cMxH > div._1Jl6c > div > div._3FkyT").text
            car_price_olx_temp.append(car_price_tag.replace("₹ ","").replace(",",""))
        WebDriverWait(driver,1)
    print(len(brand_olx_temp))
    print(len(model_olx_temp))        
    print(len(man_year_olx_temp))
    print(len(driven_km_olx_temp))
    print(len(fuel_type_olx_temp))
    print(len(owners_olx_temp))
    print(len(car_loc_olx_temp))
    print(len(car_price_olx_temp))


In [14]:
def olx():
    for brand,model,man_year,driven_km,fuel_type,owners,car_loc,car_price in zip(brand_olx_temp,model_olx_temp,man_year_olx_temp,driven_km_olx_temp,fuel_type_olx_temp,owners_olx_temp,car_loc_olx_temp,car_price_olx_temp):
        brand_olx.append(brand)
        model_olx.append(model)
        man_year_olx.append(man_year)
        driven_km_olx.append(driven_km)
        fuel_type_olx.append(fuel_type)
        owners_olx.append(owners)
        car_loc_olx.append(car_loc)
        car_price_olx.append(car_price)
    print(len(brand_olx))
    print(len(model_olx))
    print(len(man_year_olx))
    print(len(driven_km_olx))
    print(len(fuel_type_olx))
    print(len(owners_olx))
    print(len(car_loc_olx))
    print(len(car_price_olx))

In [15]:
olx_temp(0,200)

200
200
200
200
200
200
200
200


In [18]:
olx()

200
200
200
200
200
200
200
200


In [19]:
olx_temp(200,400)

200
200
200
200
200
200
200
200


In [20]:
olx()

400
400
400
400
400
400
400
400


In [21]:
olx_temp(400,600)

200
200
200
200
200
200
200
200


In [22]:
olx()

600
600
600
600
600
600
600
600


In [23]:
olx_temp(600,800)

200
200
200
200
200
200
200
200


In [24]:
olx()

800
800
800
800
800
800
800
800


In [25]:
olx_temp(800,1000)

200
200
200
200
200
200
200
200


In [26]:
olx()

1000
1000
1000
1000
1000
1000
1000
1000


In [28]:
olx_temp(1000,1200)

200
200
200
200
200
200
200
200


In [29]:
olx()

1200
1200
1200
1200
1200
1200
1200
1200


In [30]:
olx_temp(1200,1400)

200
200
200
200
200
200
200
200


In [31]:
olx()

1400
1400
1400
1400
1400
1400
1400
1400


In [33]:
olx_temp(1400,1600)

200
200
200
200
200
200
200
200


In [34]:
olx()

1600
1600
1600
1600
1600
1600
1600
1600


In [35]:
olx_temp(1600,1800)

200
200
200
200
200
200
200
200


In [36]:
olx()

1800
1800
1800
1800
1800
1800
1800
1800


In [38]:
olx_temp(1800,2000)

200
200
200
200
200
200
200
200


In [39]:
olx()

2000
2000
2000
2000
2000
2000
2000
2000


We have scraped data for 2000 used cars from Olx.in.

#### Collecting Data from Cardekho.com

In [41]:
Locations = ['Delhi', 'Noida', 'Gurgaon', 'Mumbai', 'Pune', 'Bengaluru', 'Hyderabad', 'Chennai', 'Kerala', 'Ahmedabad']
url_cardekho = []

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
driver.maximize_window()
time.sleep(1)
    
for i in Locations:
    driver.get("https://www.cardekho.com")
    time.sleep(3)
    
    #Clicking the Category - Used Car

    try:
        WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'li:nth-child(2)')))
        used_car_menu = driver.find_element_by_css_selector("div > nav > ul > li:nth-child(2) > span > span")
        used_car_menu.click()
        time.sleep(1)
    except (WebDriverException,TimeoutException) as e:
        used_car_menu = driver.find_element_by_css_selector("div > nav > ul > li:nth-child(2) > span > span")
        used_car_menu.click()
        time.sleep(1)
    

    try:
        try:        
            car_dekho_used = driver.find_element_by_xpath("//div[@id= 'rf01']/header/div[2]/div/div/nav/ul/li[2]/ul/li[2]/a/span")
            car_dekho_used.click()
            
        except (ElementNotInteractableException,NoSuchElementException,TimeoutException) as e:
            used_car_menu.click()
            time.sleep(1)
            car_dekho_used = driver.find_element_by_css_selector("li.mainMenu.current > ul > li:nth-child(2) > a > span")
            car_dekho_used.click()
    except (NoSuchElementException,WebDriverException) as e:
        time.sleep(1)
        car_dekho_used = driver.find_element_by_xpath("//div[@id= 'rf01']/header/div[2]/div/div/nav/ul/li[2]/ul/li[2]/a/span")
        car_dekho_used.click()
        
    
    #Clicking the location
    try:
        try:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div.inputfield')))
            location_button = driver.find_element_by_css_selector("div.gsc_col-xs-12.gsc_col-sm-12.gsc_col-md-4.gsc_col-lg-3.BookingfilterPanel.stickyfilter > div > div:nth-child(1) > div:nth-child(1) > div.inputfield > input[type=text]")
            location_button.click()
        except NoSuchElementException as e:
            time.sleep(1)
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div.inputfield')))
            location_button = driver.find_element_by_css_selector("div.gsc_col-xs-12.gsc_col-sm-12.gsc_col-md-4.gsc_col-lg-3.BookingfilterPanel.stickyfilter > div > div:nth-child(1) > div:nth-child(1) > div.inputfield > input[type=text]")
            location_button.click()
            
    except (ElementClickInterceptedException,NoSuchElementException,TimeoutException) as e:
        time.sleep(1)
        WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div.gsc_closeBtn')))
        driver.find_element_by_css_selector("div.gsc_row > div:nth-child(3) > div.sortByPanel.hidden-lg.hidden-md.hidden-sm.hidden-xs > div.gsc_closeBtn").click()
        location_button = driver.find_element_by_css_selector("div.gsc_col-xs-12.gsc_col-sm-12.gsc_col-md-4.gsc_col-lg-3.BookingfilterPanel.stickyfilter > div > div:nth-child(1) > div:nth-child(1) > div.inputfield > input[type=text]")
        location_button.click()
        
        
        
    #Location_input
    time.sleep(1)
    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.ID,'autocomplete')))
    location_input = driver.find_element_by_id("autocomplete")
    location_input.send_keys(i)
    
    #Selecting Location
    try:
        WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'span.pac-item-query')))
        time.sleep(1)
        select_location = driver.find_element_by_css_selector("div.pac-container.pac-logo > div:nth-child(1) > span.pac-item-query > span")
        select_location.click()
    except (NoSuchElementException,TimeoutException) as e:
        time.sleep(1)
        select_location = driver.find_element_by_css_selector("div.pac-container.pac-logo > div:nth-child(1) > span.pac-item-query > span")
        select_location.click()
    url_links_cardekho = []
    while len(url_links_cardekho)<200:
        url_links_cardekho = []
        height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, {})".format(height-1200)) 
        time.sleep(5)
        url_links = driver.find_elements_by_xpath("//div[@class = 'gsc_col-xs-7  carsName']/a")
        time.sleep(1)
        for ur in url_links:
            url_links_cardekho.append(ur.get_attribute("href"))
    url_cardekho.extend(url_links_cardekho[:200])
    
print(len(url_cardekho))

2000


We have scraped the URLs for 2000 cars for different locations from Cardekho.com. Now we can scrape the required information for each cars from the URLs.

In [42]:
driver.close()

In [43]:
brand_cardekho = []
model_cardekho = []
man_year_cardekho = []
driven_km_cardekho = []
fuel_type_cardekho = []
owners_cardekho = []
car_loc_cardekho = []
car_price_cardekho = []

brand_cardekho_temp = []
model_cardekho_temp = []
man_year_cardekho_temp = []
driven_km_cardekho_temp = []
fuel_type_cardekho_temp = []
owners_cardekho_temp = []
car_loc_cardekho_temp = []
car_price_cardekho_temp = []

In [69]:

def cardekho_temp(a,b):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")

    driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
    driver.maximize_window()
    time.sleep(2)
    
    global brand_cardekho_temp, model_cardekho_temp, man_year_cardekho_temp, driven_km_cardekho_temp, fuel_type_cardekho_temp, owners_cardekho_temp, car_loc_cardekho_temp, car_price_cardekho_temp 
    
    brand_cardekho_temp = []
    model_cardekho_temp = []
    man_year_cardekho_temp = []
    driven_km_cardekho_temp = []
    fuel_type_cardekho_temp = []
    owners_cardekho_temp = []
    car_loc_cardekho_temp = []
    car_price_cardekho_temp = []
    
    
    for i in url_cardekho[a:b]:
        if len(car_price_cardekho_temp)%100 ==0:
            time.sleep(1)
            driver.close()
            
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument("--incognito")
            driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
            driver.maximize_window()
            time.sleep(2)
        
        try:
            try:
                time.sleep(3)
                driver.get(i)
            except (WebDriverException, TimeoutException) as e:
                WebDriverWait(driver,2)
                driver.get(i)
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(3)')))
                driver.execute_script("window.scrollTo(0, 500)")
                WebDriverWait(driver,1)
        except (WebDriverException) as e:
            WebDriverWait(driver,2)
            driver.get(i)
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(3)')))
            driver.execute_script("window.scrollTo(0, 500)")
            WebDriverWait(driver,1)

        try:
            try:
                brand_tag = driver.find_element_by_xpath("//div[@class = 'paddingBorder clearfix']/div/h1").text
                keyword = re.compile(r"([A-Za-z]+)")
                brand_name = []
                brand_name = keyword.findall(str(brand_tag))
                br = []
                for i in brand_name:
                    br.extend(i.replace(" ","").split(","))
                brand_cardekho_temp.append(br[0])
            except (NoSuchElementException) as e:
                brand_cardekho_temp.append("--")
        except WebDriverException as e:
            brand_tag = driver.find_element_by_xpath("//div[@class = 'paddingBorder clearfix']/div/h1").text
            keyword = re.compile(r"([A-Za-z]+)")
            brand_name = []
            brand_name = keyword.findall(str(brand_tag))
            br = []
            for i in brand_name:
                br.extend(i.replace(" ","").split(","))
            brand_cardekho_temp.append(br[0])
                

        try:
            try:
                model_tags = driver.find_element_by_xpath("//div[@class = 'paddingBorder clearfix']/div/h1").text
                keyword = re.compile(r"[0-9]+ ")
                model_name = keyword.sub("",str(model_tags))
                model_cardekho_temp.append(model_name.replace(br[0],"")[1:])
            except (NoSuchElementException) as e:
                model_cardekho_temp.append("--")
        except WebDriverException as e:
            model_tags = driver.find_element_by_xpath("//div[@class = 'paddingBorder clearfix']/div/h1").text
            keyword = re.compile(r"[0-9]+ ")
            model_name = keyword.sub("",str(model_tags))
            model_cardekho_temp.append(model_name.replace(br[0],"")[1:])
                
       
        try:
            try:
                man_year_tags = driver.find_element_by_css_selector("ul > li:nth-child(1) > div > div.fontweight500.value").text
                man_year_cardekho_temp.append(man_year_tags)
            except (NoSuchElementException) as e:
                man_year_cardekho_temp.append("--")
        except WebDriverException as e:
            man_year_tags = driver.find_element_by_css_selector("ul > li:nth-child(1) > div > div.fontweight500.value").text
            man_year_cardekho_temp.append(man_year_tags)
            
        try:
            try:
                driven_km_tags = driver.find_element_by_css_selector(" div > ul > li:nth-child(4) > div > div.fontweight500.value").text
                driven_km_cardekho_temp.append(driven_km_tags.replace(" Kms","").replace(",",""))
            except (NoSuchElementException) as e:
                driven_km_cardekho_temp.append("--")
        except WebDriverException as e:
            driven_km_tags = driver.find_element_by_css_selector(" div > ul > li:nth-child(4) > div > div.fontweight500.value").text
            driven_km_cardekho_temp.append(driven_km_tags.replace(" Kms","").replace(",",""))
            
                
        try:
            try:
                fuel_type_tags = driver.find_element_by_css_selector("ul > li:nth-child(3) > div > div.fontweight500.value").text
                fuel_type_cardekho_temp.append(fuel_type_tags)
            except (NoSuchElementException) as e:
                fuel_type_cardekho_temp.append("--")
        except WebDriverException as e:
            fuel_type_tags = driver.find_element_by_css_selector("ul > li:nth-child(3) > div > div.fontweight500.value").text
            fuel_type_cardekho_temp.append(fuel_type_tags)
        
        try:
            try:
                owner_tags = driver.find_element_by_css_selector("ul > li:nth-child(6) > div > div.fontweight500.value").text
                owners_cardekho_temp.append(owner_tags)
            except (NoSuchElementException) as e:
                owners_cardekho_temp.append("--")
        except WebDriverException as e:
            owner_tags = driver.find_element_by_css_selector("ul > li:nth-child(6) > div > div.fontweight500.value").text
            owners_cardekho_temp.append(owner_tags)
        
        try:
            try:
                car_loc_tag = driver.find_element_by_css_selector("div.uclocationAddress.ucStick-uclocationAddress.gsc_col-xs-12 > div.ucaddressline.posR > span.uc-add-text > span:nth-child(2)").text
                car_loc_cardekho_temp.append(car_loc_tag)
            except (NoSuchElementException) as e:
                car_loc_cardekho_temp.append("--")
        except WebDriverException as e:
            car_loc_tag = driver.find_element_by_css_selector("div.uclocationAddress.ucStick-uclocationAddress.gsc_col-xs-12 > div.ucaddressline.posR > span.uc-add-text > span:nth-child(2)").text
            car_loc_cardekho_temp.append(car_loc_tag)

        try:
            try:
                car_price_tag = driver.find_element_by_css_selector("div.paddingBorder.clearfix > div:nth-child(2) > div.left-side.gsc_col-xs-7 > div.priceSection > span:nth-child(2)").text
                car_price_cardekho_temp.append(car_price_tag.replace(",",""))
            except (NoSuchElementException) as e:
                car_price_cardekho_temp.append("--")
        except WebDriverException as e:
            car_price_tag = driver.find_element_by_css_selector("div.paddingBorder.clearfix > div:nth-child(2) > div.left-side.gsc_col-xs-7 > div.priceSection > span:nth-child(2)").text
            car_price_cardekho_temp.append(car_price_tag.replace(",",""))
        time.sleep(1)
    print(len(brand_cardekho_temp))
    print(len(model_cardekho_temp))        
    print(len(man_year_cardekho_temp))
    print(len(fuel_type_cardekho_temp))
    print(len(owners_cardekho_temp))
    print(len(car_loc_cardekho_temp))
    print(len(car_price_cardekho_temp))

In [70]:
def cardekho():
    for brand,model,man_year,driven_km,fuel_type,owners,car_loc,car_price in zip(brand_cardekho_temp,model_cardekho_temp,man_year_cardekho_temp,driven_km_cardekho_temp,fuel_type_cardekho_temp,owners_cardekho_temp,car_loc_cardekho_temp,car_price_cardekho_temp):
        brand_cardekho.append(brand)
        model_cardekho.append(model)
        man_year_cardekho.append(man_year)
        driven_km_cardekho.append(driven_km)
        fuel_type_cardekho.append(fuel_type)
        owners_cardekho.append(owners)
        car_loc_cardekho.append(car_loc)
        car_price_cardekho.append(car_price)
    print(len(brand_cardekho))
    print(len(model_cardekho))
    print(len(man_year_cardekho))
    print(len(driven_km_cardekho))
    print(len(fuel_type_cardekho))
    print(len(owners_cardekho))
    print(len(car_loc_cardekho))
    print(len(car_price_cardekho))

In [72]:
cardekho_temp(0,500)

500
500
500
500
500
500
500


In [73]:
cardekho()

500
500
500
500
500
500
500
500


In [74]:
cardekho_temp(500,1000)

500
500
500
500
500
500
500


In [75]:
cardekho()

1000
1000
1000
1000
1000
1000
1000
1000


In [76]:
cardekho_temp(1000,1500)

500
500
500
500
500
500
500


In [77]:
cardekho()

1500
1500
1500
1500
1500
1500
1500
1500


In [79]:
cardekho_temp(1500,2000)

500
500
500
500
500
500
500


In [80]:
cardekho()

2000
2000
2000
2000
2000
2000
2000
2000


We have scraped data for 2000 used cars from Cardekho.com

#### Collecting data from Cars24.com

In [86]:
Locations = ['Delhi', 'Noida', 'Gurgaon', 'Mumbai', 'Pune', 'Bengaluru', 'Hyderabad', 'Chennai', 'Ahmedabad']
url_cars24 = []

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
driver.maximize_window()
time.sleep(1)

for i in Locations:
    driver.get('https://www.cars24.com')
    time.sleep(2)

    #Manual Selection
    try:
        manual_loc = driver.find_element_by_css_selector("div.rk1q-.Komfq > div._1K8Qe > button")
        manual_loc.click()
        time.sleep(1)
    except NoSuchElementException:
        manual_loc = driver.find_element_by_css_selector("div._38_So > header > div > div._1GRL2 > div:nth-child(2) > p")
        manual_loc.click()
        time.sleep(1)

    #Input
    location_input = driver.find_element_by_css_selector("div.container._3TGxN > div > div._2rz1w > div > input")
    location_input.send_keys(i)
    time.sleep(1)

    #Selecting the location
    try:
        loc_selection = driver.find_element_by_css_selector("div._3ZOW1 > div > div > div > div > div:nth-child(3) > ul > li > a > h6")
        loc_selection.click()
    except NoSuchElementException:
        loc_selection = driver.find_element_by_css_selector("div._3ZOW1 > div > div > div > div > div:nth-child(3) > div > div > ul > div > li > a > span")
        loc_selection.click()
        
    #Category - View all cars
    time.sleep(2)
    view_cars = driver.find_element_by_css_selector("div > div.ITdIr > div._175lW > button")
    view_cars.click()

    time.sleep(2)
    url_links_cars24 = []
    while len(url_links_cars24)<200:
        url_links_cars24 = []
        height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, {})".format(height-2300)) 
        time.sleep(3)
        url_links = driver.find_elements_by_xpath("//div[@class = 'o0pOR']/div[3]/div/div/div/a")
        time.sleep(1)
        for ur in url_links:
            url_links_cars24.append(ur.get_attribute("href"))
    url_cars24.extend(url_links_cars24[:200])
len(url_cars24)

1800

We have scraped the URLs for 1800 cars for different locations from Cars24.com. Now we can scrape the required information for each cars from the URLs.

In [87]:
driver.close()

In [96]:
brand_cars24 = []
model_cars24 = []
man_year_cars24 = []
driven_km_cars24 = []
fuel_type_cars24 = []
owners_cars24 = []
car_loc_cars24 = []
car_price_cars24 = []

brand_cars24_temp = []
model_cars24_temp = []
man_year_cars24_temp = []
driven_km_cars24_temp = []
fuel_type_cars24_temp = []
owners_cars24_temp = []
car_loc_cars24_temp = []
car_price_cars24_temp = []

In [239]:

def cars24_temp(a,b):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")

    driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
    driver.maximize_window()
    
    global brand_cars24_temp, model_cars24_temp, man_year_cars24_temp, driven_km_cars24_temp, fuel_type_cars24_temp, owners_cars24_temp, car_loc_cars24_temp, car_price_cars24_temp
    
    brand_cars24_temp = []
    model_cars24_temp = []
    man_year_cars24_temp = []
    driven_km_cars24_temp = []
    fuel_type_cars24_temp = []
    owners_cars24_temp = []
    car_loc_cars24_temp = []
    car_price_cars24_temp = []


    for i in url_cars24[a:b]:
        if len(car_price_cars24_temp)%200 ==0:
            driver.close()
            
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument("--incognito")
            driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
            driver.maximize_window()
            time.sleep(2)
      
        try:
            try:
                time.sleep(1)
                driver.get(i)
                time.sleep(1)
            except (WebDriverException, TimeoutException) as e:
                WebDriverWait(driver,2)
                driver.get(i)
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
                driver.execute_script("window.scrollTo(0, 500)")
                WebDriverWait(driver,1)
        except (WebDriverException) as e:
            WebDriverWait(driver,2)
            driver.get(i)
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
            driver.execute_script("window.scrollTo(0, 500)")
            WebDriverWait(driver,1)
        
        try:
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
            location_popup = driver.find_element_by_xpath("/html/body/div[1]/div[7]/div/div/h3/div/img")
            location_popup.click()
        except (NoSuchElementException,TimeoutException,WebDriverException) as e:
            pass
        try:
            try:
                brand_tag = driver.find_element_by_css_selector("div.NCchD > div > div > div > div._187DF > div._10kZS > h2").text
                keyword = re.compile(r"([A-Za-z]+)")
                brand_name = []
                brand_name = keyword.findall(str(brand_tag))
                br = []
                for i in brand_name:
                    br.extend(i.replace(" ","").split(","))
                brand_cars24_temp.append(br[0])
            except (NoSuchElementException,TimeoutException) as e:
                brand_cars24_temp.append("--")
        except (WebDriverException, StaleElementReferenceException) as e: 
            brand_tag = driver.find_element_by_css_selector("div.NCchD > div > div > div > div._187DF > div._10kZS > h2").text
            keyword = re.compile(r"([A-Za-z]+)")
            brand_name = []
            brand_name = keyword.findall(str(brand_tag))
            br = []
            for i in brand_name:
                br.extend(i.replace(" ","").split(","))
            brand_cars24_temp.append(br[0])
            
        try:
            try:
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div._10kZS')))
                model_tags = driver.find_element_by_css_selector("body > div.js-content > div.NCchD > div > div > div > div._187DF > div._10kZS > h2").text
                keyword = re.compile(r"[0-9]+ ")
                model_name = keyword.sub("",str(model_tags))
                model_cars24_temp.append(model_name.replace(br[0],"")[1:])
            except (NoSuchElementException,TimeoutException) as e:
                model_cars24_temp.append("--")
        except (WebDriverException, StaleElementReferenceException) as e: 
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
            model_tags = driver.find_element_by_css_selector("body > div.js-content > div.NCchD > div > div > div > div._187DF > div._10kZS > h2").text
            keyword = re.compile(r"[0-9]+ ")
            model_name = keyword.sub("",str(model_tags))
            model_cars24_temp.append(model_name.replace(br[0],"")[1:])

        try:
            try:
                man_year_tags = driver.find_element_by_css_selector("div.NCchD > div > div > div > div._187DF > div._10kZS > h2").text
                man_year_cars24_temp.append(man_year_tags[0:4])
            except (NoSuchElementException,TimeoutException) as e:
                man_year_cars24_temp.append("--")
        except (WebDriverException, StaleElementReferenceException) as e: 
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
            man_year_tags = driver.find_element_by_css_selector("div.NCchD > div > div > div > div._187DF > div._10kZS > h2").text
            man_year_cars24_temp.append(man_year_tags[0:4])
        
        try:
            try:
                driven_km_tags = driver.find_element_by_css_selector("div._187DF > div._10kZS > ul > li:nth-child(1)").text
                driven_km_cars24_temp.append(driven_km_tags.replace(" km",""))
            except (NoSuchElementException,TimeoutException) as e:
                driven_km_cars24_temp.append("--")
        except (WebDriverException, StaleElementReferenceException) as e: 
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
            driven_km_tags = driver.find_element_by_css_selector("div._187DF > div._10kZS > ul > li:nth-child(1)").text
            driven_km_cars24_temp.append(driven_km_tags.replace(" km",""))
        
        try:
            try:
                fuel_type_tags = driver.find_element_by_css_selector("div._187DF > div._10kZS > ul > li:nth-child(3)").text
                fuel_type_cars24_temp.append(fuel_type_tags)
            except (NoSuchElementException,TimeoutException) as e:
                fuel_type_cars24_temp.append("--")
        except (WebDriverException, StaleElementReferenceException) as e: 
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
            fuel_type_tags = driver.find_element_by_css_selector("div._187DF > div._10kZS > ul > li:nth-child(3)").text
            fuel_type_cars24_temp.append(fuel_type_tags)

        try:
            try:
                owner_tags = driver.find_element_by_css_selector("div._187DF > div._10kZS > ul > li:nth-child(5)").text
                owners_cars24_temp.append(owner_tags)
            except (NoSuchElementException,TimeoutException) as e:
                owners_cars24_temp.append("--")
        except (WebDriverException, StaleElementReferenceException) as e: 
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
            owner_tags = driver.find_element_by_css_selector("div._187DF > div._10kZS > ul > li:nth-child(5)").text
            owners_cars24_temp.append(owner_tags)
        
        try:
            try:
                car_loc_tag = driver.find_element_by_css_selector("div._10kZS > ul > li:nth-child(7)").text
                car_loc_cars24_temp.append(car_loc_tag)
            except (NoSuchElementException,TimeoutException) as e:
                car_loc_cars24_temp.append("--")
        except (WebDriverException, StaleElementReferenceException) as e: 
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
            car_loc_tag = driver.find_element_by_css_selector("div._10kZS > ul > li:nth-child(7)").text
            car_loc_cars24_temp.append(car_loc_tag)
        
        try:
            try:
                car_price_tag = driver.find_element_by_xpath("//strong[@class = '_2yYvS Wqfj6']").text
                keyword = re.compile(r"\n+(₹[0-9,]+)")
                car_price_info = keyword.findall(car_price_tag)
                for car in car_price_info:
                    car_price_cars24_temp.append(car.replace("₹","").replace(",",""))
            except (NoSuchElementException,TimeoutException) as e:
                car_price_cars24_temp.append("--")
        except (WebDriverException, StaleElementReferenceException) as e: 
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div:nth-child(1)')))
            car_price_tag = driver.find_element_by_xpath("//strong[@class = '_2yYvS Wqfj6']").text
            keyword = re.compile(r"\n+(₹[0-9,]+)")
            car_price_info = keyword.findall(car_price_tag)
            for car in car_price_info:
                car_price_cars24_temp.append(car.replace("₹","").replace(",",""))
        time.sleep(1)
    print(len(brand_cars24_temp))
    print(len(model_cars24_temp))        
    print(len(man_year_cars24_temp))
    print(len(driven_km_cars24_temp))
    print(len(fuel_type_cars24_temp))
    print(len(owners_cars24_temp))
    print(len(car_loc_cars24_temp))
    print(len(car_price_cars24_temp))

In [230]:
def cars24():
    for brand,model,man_year,driven_km,fuel_type,owners,car_loc,car_price in zip(brand_cars24_temp,model_cars24_temp,man_year_cars24_temp,driven_km_cars24_temp,fuel_type_cars24_temp,owners_cars24_temp,car_loc_cars24_temp,car_price_cars24_temp):
        brand_cars24.append(brand)
        model_cars24.append(model)
        man_year_cars24.append(man_year)
        driven_km_cars24.append(driven_km)
        fuel_type_cars24.append(fuel_type)
        owners_cars24.append(owners)
        car_loc_cars24.append(car_loc)
        car_price_cars24.append(car_price)
    print(len(brand_cars24))
    print(len(model_cars24))
    print(len(man_year_cars24))
    print(len(driven_km_cars24))
    print(len(fuel_type_cars24))
    print(len(owners_cars24))
    print(len(car_loc_cars24))
    print(len(car_price_cars24))

We have scraped data for 1800 used cars from Cars24.com.

In [194]:
cars24_temp(0,200)

200
200
200
200
200
200
200
200


In [195]:
cars24()

200
200
200
200
200
200
200
200


In [196]:
cars24_temp(200,400)

200
200
200
200
200
200
200
200


In [197]:
cars24()

400
400
400
400
400
400
400
400


In [211]:
cars24_temp(400,600)

200
200
200
200
200
200
200
200


In [212]:
cars24()

600
600
600
600
600
600
600
600


In [213]:
cars24_temp(600,800)

200
200
200
200
200
200
200
200


In [214]:
cars24()

800
800
800
800
800
800
800
800


In [222]:
cars24_temp(800,1000)

200
200
200
200
200
200
200
200


In [223]:
cars24()

1000
1000
1000
1000
1000
1000
1000
1000


In [241]:
cars24_temp(1000,1200)

200
200
200
200
200
200
200
200


In [242]:
cars24()

1200
1200
1200
1200
1200
1200
1200
1200


In [248]:
cars24_temp(1200,1400)

200
200
200
200
200
200
200
200


In [249]:
cars24()

1400
1400
1400
1400
1400
1400
1400
1400


In [250]:
cars24_temp(1400,1600)

200
200
200
200
200
200
200
200


In [251]:
cars24()

1600
1600
1600
1600
1600
1600
1600
1600


In [252]:
cars24_temp(1600,1800)

200
200
200
200
200
200
200
200


In [253]:
cars24()

1800
1800
1800
1800
1800
1800
1800
1800


We have collected Used car data for 1800 cars from Cars24.com.

Now we can create the dataframe with the collected data.

In [255]:
brand = []
model = []
man_year = []
driven_km = []
fuel_type = []
owners = []
car_loc = []
car_price = []

for b,m,my,d,f,o,c,p in zip(brand_olx,model_olx,man_year_olx,driven_km_olx,fuel_type_olx,owners_olx,car_loc_olx,car_price_olx):
    brand.append(b)
    model.append(m)
    man_year.append(my)
    driven_km.append(d)
    fuel_type.append(f)
    owners.append(o)
    car_loc.append(c)
    car_price.append(p)
    
for b,m,my,d,f,o,c,p in zip(brand_cardekho,model_cardekho,man_year_cardekho,driven_km_cardekho,fuel_type_cardekho,owners_cardekho,car_loc_cardekho,car_price_cardekho):
    brand.append(b)
    model.append(m)
    man_year.append(my)
    driven_km.append(d)
    fuel_type.append(f)
    owners.append(o)
    car_loc.append(c)
    car_price.append(p)
    
for b,m,my,d,f,o,c,p in zip(brand_cars24,model_cars24,man_year_cars24,driven_km_cars24,fuel_type_cars24,owners_cars24,car_loc_cars24,car_price_cars24):
    brand.append(b)
    model.append(m)
    man_year.append(my)
    driven_km.append(d.replace(",",""))
    fuel_type.append(f)
    owners.append(o)
    car_loc.append(c)
    car_price.append(p)



print(len(brand))
print(len(model))
print(len(man_year))
print(len(driven_km))
print(len(fuel_type))
print(len(owners))
print(len(car_loc))
print(len(car_price))

5800
5800
5800
5800
5800
5800
5800
5800


In [259]:
#DataFrame
used_cars = pd.DataFrame({})
used_cars['Brand'] = brand
used_cars['Model'] = model
used_cars['Manufacturing_Year'] = man_year
used_cars['Kilometers_Driven'] = driven_km
used_cars['Fuel_Type'] = fuel_type
used_cars['No_of_owners'] = owners
used_cars['Location'] = car_loc
used_cars['Car_Price'] = car_price

used_cars

,Brand,Model,Manufacturing_Year,Kilometers_Driven,Fuel_Type,No_of_owners,Location,Car_Price
0,Skoda,Superb,2012,58000.0,DIESEL,1st,"Ramesh Nagar, Delhi, Delhi",425000
1,Benz,ercedes- S-Class,2016,41000.0,PETROL,1st,"Tilak Nagar, Delhi, Delhi",6990000
2,Benz,ercedes- C-Class,2021,9700.0,PETROL,1st,"F 10, Shopping Centre, Near OBC Bank, Mansarov...",4990000
3,Skoda,Superb,2010,120000,PETROL,First,--,165000
4,Maruti,Suzuki Swift Dzire,2016,70000,CNG,Second,--,395000
...,...,...,...,...,...,...,...,...
5795,Maruti,S PRESSO VXI MANUAL,2020,18681,Petrol,1st Owner,GJ-18,439699
5796,Maruti,IGNIS ZETA 1.KMANUAL,2018,48213,Petrol,1st Owner,GJ-15,474799
5797,Maruti,Celerio X Zxi AMT AUTOMATIC,2019,67192,Petrol,1st Owner,GJ-06,508199
5798,Jeep,Compass LIMITED 1.AT AUTOMATIC,2017,63038,Petrol,1st Owner,GJ-23,1125799


We can see that there are many rows with no values in all the columns, so we can drop them before saving the dataset.

In [264]:
import numpy as np
used_cars.replace("--",np.nan,inplace = True)
used_cars = used_cars.dropna(subset=['Car_Price'])
used_cars= used_cars.reset_index(drop = True)

In [265]:
used_cars

,Brand,Model,Manufacturing_Year,Kilometers_Driven,Fuel_Type,No_of_owners,Location,Car_Price
0,Skoda,Superb,2012,58000.0,DIESEL,1st,"Ramesh Nagar, Delhi, Delhi",425000
1,Benz,ercedes- S-Class,2016,41000.0,PETROL,1st,"Tilak Nagar, Delhi, Delhi",6990000
2,Benz,ercedes- C-Class,2021,9700.0,PETROL,1st,"F 10, Shopping Centre, Near OBC Bank, Mansarov...",4990000
3,Skoda,Superb,2010,120000,PETROL,First,NaN,165000
4,Maruti,Suzuki Swift Dzire,2016,70000,CNG,Second,NaN,395000
...,...,...,...,...,...,...,...,...
5265,Maruti,S PRESSO VXI MANUAL,2020,18681,Petrol,1st Owner,GJ-18,439699
5266,Maruti,IGNIS ZETA 1.KMANUAL,2018,48213,Petrol,1st Owner,GJ-15,474799
5267,Maruti,Celerio X Zxi AMT AUTOMATIC,2019,67192,Petrol,1st Owner,GJ-06,508199
5268,Jeep,Compass LIMITED 1.AT AUTOMATIC,2017,63038,Petrol,1st Owner,GJ-23,1125799


In [267]:
#Saving the dataset
used_cars.to_csv("used_cars.csv",index=False)

We have collected 5270 used car data for building the machine learning model that can make the predictions.